## Część 2

In [5]:
import os
import pandas as pd
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="secret.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()
query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)
query_result = query_job.result()
df1 = query_result.to_dataframe()
df1

E:\Python Projekty\BigData1\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AD,2022-02-22,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
1,AD,2022-04-18,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
2,AD,2020-01-13,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
3,AD,2022-04-24,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
4,AD,2020-08-03,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
5,AD,2021-05-30,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
6,AD,2021-03-26,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
7,AD,2020-08-23,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
8,AD,2021-07-24,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
9,AD,2022-05-16,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)


## Część 3

3.1. Sprawdź, ile jest zapisanych wierszy z danymi. 

In [6]:
query = ("""
select count(*) from bigquery-public-data.covid19_open_data.covid19_open_data;
""")

query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

E:\Python Projekty\BigData1\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,f0_
0,22756333


 3.2. Sprawdź, ile krajów jest uwzględnionych w danych. 



In [1]:
query = ("""
select count(*) FROM (SELECT DISTINCT country_name FROM bigquery-public-data.covid19_open_data.covid19_open_data);
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

NameError: name 'client' is not defined

 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.



In [10]:
df1[df1.country_name == 'Andorra']
# do dopytania

,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AD,2022-02-22,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
1,AD,2022-04-18,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
2,AD,2020-01-13,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
3,AD,2022-04-24,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
4,AD,2020-08-03,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
5,AD,2021-05-30,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
6,AD,2021-03-26,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
7,AD,2020-08-23,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
8,AD,2021-07-24,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
9,AD,2022-05-16,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)


 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.  



In [8]:
# chodzi po prostu o data type? float64 i int64

 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.



In [13]:
query = ("""
select min(date) as poczatek_przedzialu, max(date) as koniec_przedzialu FROM bigquery-public-data.covid19_open_data.covid19_open_data;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df
# co to znaczy nowych?

E:\Python Projekty\BigData1\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,poczatek_przedzialu,koniec_przedzialu
0,2020-01-01,2022-09-17


 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 



In [ ]:
# ile jest kolumn
# ile jest pustych wartosci w poszczegolnych kolumnach / w ktorych kolumnach jest ponad 80% nulli
# w jakim formacie zapisana jest data
# jaki jest separator dziesietny
# w jakim formacie zapisany sa wartosci w location_key

## Część 4

4.1. Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.



In [ ]:
# nazwa kraju, populacja, liczba kobiet i mezczyzn, liczba ludnosci w wieku najwiekszego ryzyka (60+)